In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

In [ ]:
load_dotenv()

checkpointer = MemorySaver()

model = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

In [ ]:
class StateMessage(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
def chat_node(state:StateMessage) -> StateMessage:

    messages = state['messages']
    response = model.invoke(messages)
    return {'messages':[response]}

In [ ]:
graph = StateGraph(StateMessage)

graph.add_node('chat_node',chat_node)

graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

workflow = graph.compile(checkpointer=checkpointer)

In [ ]:
initial_state = {
    'messages':[HumanMessage(content='wassup buddy ?')]
}
workflow.invoke(initial_state)

In [ ]:
thread_id = 1

while True:
    user_input = input('Type here: ')
    print('User: ',user_input)
    if user_input == 'exit':
        break

    config = {'configurable':{'thread_id':thread_id}}
    response = workflow.invoke(
        {'messages':[HumanMessage(content=user_input)]},
        config=config
    )

    print('AI: ',response['messages'][-1].content)